In [2]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual,fixed
import ipywidgets as widgets

import pandas as pd

In [3]:
import matplotlib.pyplot as plt

In [56]:
df2=pd.read_csv("demographics.csv",header=0,na_values=['(NA)']).fillna(0)
df3=pd.read_csv("demographics2.csv",header=0,na_values=['(NA)']).fillna(0)
df3=df3.iloc[1:,:]
df3['FIPS']=df3['FIPS'].astype('float')
df2=df2.iloc[1:,:]
df2['FIPS']=df2['FIPS'].astype('float')
df_demographics=pd.merge(df2,df3,on='FIPS')
percent_young=df_demographics.iloc[:,9:18].astype('float').sum(axis=1)
percent_old=100-percent_young
df_demographics.insert(7,'% Population 64 years and younger',percent_young)
df_demographics.insert(8,'% Population > 64 years',percent_old)






def data_from_date(date):
    url='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'+date+'-2020.csv'
    df_jh=pd.read_csv(url)
    df_merged=pd.merge(df_jh,df_demographics,on='FIPS')
    cols=df_merged.columns[5:]
    df_merged[cols] = df_merged[cols].apply(pd.to_numeric, errors='coerce')
    df_merged.rename(columns={"Admin2":"County",'Long_':"Longitude",'Lat':'Latitude'},inplace=True)
    df_merged.set_index('County',inplace=True)
    df_merged.rename(columns={"Province_State":"State"},inplace=True)
    df_merged.insert(6, 'Confirmed cases per 100,000 residents', (df_merged['Confirmed'].div(df_merged['Total Population'])*100000).astype('int'))
    df_merged.insert(8, 'Deaths per 100,000 residents', (df_merged['Deaths'].div(df_merged['Total Population'])*100000).astype('int'))


    return(df_merged)

def plot_time_series(df,list_of_counties,per_capita=True):
    if per_capita:
        ax=df.loc[list_of_counties].iloc[:,11:].div(dfs['04-01'].loc[list_of_counties,'Total Population'].values,axis=0).T.plot()
        ax.set_ylabel("Cumulative cases per capita")
    else:
        ax=df.loc[list_of_counties].iloc[:,11:].T.plot()
        ax.set_ylabel("Cumulative cases")


In [57]:
dates=['04-01','04-15','05-01','05-15','06-01','06-15','07-01','07-15','07-25']
dfs={}
for index, this_date in enumerate(dates):
    print(this_date)
    dfs[this_date]=data_from_date(this_date)

04-01


04-15


05-01


05-15


06-01


06-15


07-01


07-15


07-25


In [60]:
pd.to_pickle(dfs,'dfs.pkl')

In [59]:
dfs['04-01'].columns[1:22]

Index(['State', 'Country_Region', 'Last_Update', 'Latitude', 'Longitude',
       'Confirmed cases per 100,000 residents', 'Confirmed',
       'Deaths per 100,000 residents', 'Deaths', 'Recovered', 'Active',
       'Combined_Key', 'Name of Area', 'Qualifying Name',
       'State/U.S.-Abbreviation (USPS)', 'Total Population',
       'Total Population.1', 'Population Density (Per Sq. Mile)',
       '% Population 64 years and younger', '% Population > 64 years',
       'Area (Land)'],
      dtype='object')

Look at the data for Yuba County, California

In [16]:
def make_plots(date,yaxis, xaxis,dfs):
    print(f"Correlation between {xaxis} and {yaxis}: {dfs[date][xaxis].corr(dfs[date][yaxis])}")
    dfs[date].plot(xaxis,yaxis,kind='scatter')
    plt.show()
    return  dfs[date][[xaxis,yaxis]].hist()
kwargs = {'date':dfs.keys(),'yaxis':list(dfs['04-01'].select_dtypes('number').columns[2:]),'xaxis':list(dfs['04-01'].select_dtypes('number').columns[2:]),'dfs':fixed(dfs)}

In [28]:
dfs['07-25'].iloc[:,23:32].sum(axis=1)

County
Alameda     86.87
Alpine      74.43
Amador      73.69
Anderson    85.83
Andrews     89.66
            ...  
Yolo        88.16
Young       80.21
Yuba        87.98
Zapata      86.10
Zavala      86.28
Length: 308, dtype: float64

In [17]:
interact(make_plots,**kwargs)


interactive(children=(Dropdown(description='date', options=('04-01', '04-15', '05-01', '05-15', '06-01', '06-1…

<function __main__.make_plots(date, yaxis, xaxis, dfs)>

In [36]:
interact(make_plots,**kwargs)

interactive(children=(Dropdown(description='date', options=('04-01', '04-15', '05-01', '05-15', '06-01', '06-1…

<function __main__.make_plots(date, yaxis, xaxis)>

In [61]:
df_timeseries=pd.read_csv('https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
groups=df_timeseries.groupby('Province_State')
df_texas=groups.get_group('Texas').copy()
df_texas.rename(columns={'Admin2':'County'},inplace=True)
df_texas.set_index('County',inplace=True)
df_california=groups.get_group('California').copy()
df_california.rename(columns={'Admin2':'County'},inplace=True)
df_california.set_index('County',inplace=True)

In [69]:
pd.to_pickle(df_texas,'texas.pkl')
df_california.to_csv('california.csv')
pd.to_pickle(df_california,'california.pkl')

In [12]:
df_california

,UID,iso2,iso3,code3,FIPS,Province_State,Country_Region,Lat,Long_,Combined_Key,...,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20
County,,,,,,,,,,,,,,,,,,,,,
Alameda,84006001,US,USA,840,6001.0,California,US,37.646294,-121.892927,"Alameda, California, US",...,9110,9237,9256,9383,9643,9869,10214,10330,10330,10438
Alpine,84006003,US,USA,840,6003.0,California,US,38.596786,-119.822359,"Alpine, California, US",...,2,2,2,2,2,2,2,2,2,2
Amador,84006005,US,USA,840,6005.0,California,US,38.445831,-120.656960,"Amador, California, US",...,51,54,57,62,65,65,71,71,76,81
Butte,84006007,US,USA,840,6007.0,California,US,39.667278,-121.600525,"Butte, California, US",...,522,522,640,658,683,722,758,758,758,820
Calaveras,84006009,US,USA,840,6009.0,California,US,38.205371,-120.552913,"Calaveras, California, US",...,87,87,87,94,94,94,99,99,99,99
Colusa,84006011,US,USA,840,6011.0,California,US,39.178820,-122.233173,"Colusa, California, US",...,173,173,223,234,242,250,263,263,263,273
Contra Costa,84006013,US,USA,840,6013.0,California,US,37.919235,-121.928953,"Contra Costa, California, US",...,5538,5606,5731,5931,6202,6425,6627,6770,6933,7073
Del Norte,84006015,US,USA,840,6015.0,California,US,41.742283,-123.897406,"Del Norte, California, US",...,66,66,66,68,79,79,87,87,87,87
El Dorado,84006017,US,USA,840,6017.0,California,US,38.779660,-120.523317,"El Dorado, California, US",...,394,394,443,464,486,508,513,513,513,513
